<a href="https://colab.research.google.com/github/rachanagb/Cricket-Analysis-using-YOLOv8n/blob/main/yolomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics opencv-python numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.9 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")
video_path = "net_session.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise IOError("Cannot open input video!")

fps  = int(cap.get(cv2.CAP_PROP_FPS))
w    = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h    = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(fps)
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Total frames in video:", total_frames)

out = cv2.VideoWriter(
    "output_analysis.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (w, h)
)


while True:
    ret, frame = cap.read()
    if not ret:
        break
    crop_y1 = int(h * 0.15)
    crop_y2 = int(h * 0.85)
    crop_x1 = int(w * 0.3)
    crop_x2 = int(w * 0.7)
    cropped_frame = frame[crop_y1:crop_y2, crop_x1:crop_x2]

    results = model(cropped_frame, verbose=False)
    annotated_crop = cropped_frame.copy()
    if len(results) > 0 and results[0].keypoints is not None:
        annotated_crop = results[0].plot()
    annotated = frame.copy()
    annotated[crop_y1:crop_y2, crop_x1:crop_x2] = annotated_crop
    out.write(annotated)
cap.release()
out.release()
cv2.destroyAllWindows()

print("output_analysis.mp4 saved successfully")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
29
Total frames in video: 18007
output_analysis.mp4 saved successfully


In [ ]:
!ffmpeg -i output_analysis.mp4 -c:v libx264 -preset ultrafast -acodec aac fixed_output.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO


model = YOLO("yolov8n-pose.pt")


video_path = "fixed_output.mp4"
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise IOError("Cannot open video file!")


def get_stance_from_keypoints(kp):

    left_shoulder_x, right_shoulder_x = kp[5][0], kp[6][0]
    left_knee_x, right_knee_x = kp[11][0], kp[12][0]

    delta_s = right_shoulder_x - left_shoulder_x
    delta_k = right_knee_x - left_knee_x

    if delta_s < 0 and delta_k < 0:
        return "Right-Handed"
    elif delta_s > 0 and delta_k > 0:
        return "Left-Handed"
    else:
        return "Uncertain"

target_frames = [500,10500]
frame_results = {}


for idx in target_frames:
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
    ret, frame = cap.read()
    if not ret:
        print(f"Could not read frame {idx}")
        continue

    results = model(frame, verbose=False)

    if len(results) == 0 or results[0].boxes is None or results[0].keypoints is None:
        print(f"No detections in frame {idx}")
        continue

    boxes = results[0].boxes.xyxy
    keypoints = results[0].keypoints.xy

    if len(boxes) == 0:
        print(f"No detections in frame {idx}")
        continue

    areas = ((boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])).cpu().numpy()
    batsman_idx = np.argmin(areas)

    kp = keypoints[batsman_idx].cpu().numpy()
    stance = get_stance_from_keypoints(kp)
    frame_results[idx] = stance

for stance in frame_results.values():
    print(f"Batsman stance: {stance}")

cap.release()
cv2.destroyAllWindows()



Batsman stance: Left-Handed
Batsman stance: Right-Handed
